### 🔧 环境配置和检查

#### 概述

本教程需要特定的环境配置以确保最佳学习体验。以下配置将帮助你：

- 使用统一的conda环境：激活统一的学习环境
- 通过国内镜像源快速安装依赖：配置pip使用清华镜像源
- 加速模型下载：设置HuggingFace镜像代理
- 检查系统配置：检查硬件和软件配置

#### 配置

- **所需环境及其依赖已经部署好**
- 在`Notebook`右上角选择`jupyter内核`为`python(flyai_agent_in_action)`，即可执行下方代码

In [1]:
%%script bash

# 1. 激活 conda 环境 (仅对当前单元格有效)
eval "$(conda shell.bash hook)"
conda activate agent101

echo "========================================="
echo "== Conda 环境检查报告 (仅针对当前 Bash 子进程) =="
echo "========================================="

# 2. 检查当前激活的环境
CURRENT_ENV_NAME=$(basename $CONDA_PREFIX)

if [ "$CURRENT_ENV_NAME" = "agent101" ]; then
    echo "✅ 当前单元格已成功激活到 agent101 环境。"
    echo "✅ 正在使用的环境路径: $CONDA_PREFIX"
    echo ""
    echo "💡 提示: 后续的Python单元格将使用Notebook当前选择的Jupyter内核。"
    echo "   如果需要后续单元格也使用此环境，请执行以下操作:"
    echo "   1. 检查 Notebook 右上角是否已选择 'python(agent101)'。"
else
    echo "❌ 激活失败或环境名称不匹配。当前环境: $CURRENT_ENV_NAME"
    echo ""
    echo "⚠️ 严重提示: 建议将 Notebook 的 Jupyter **内核 (Kernel)** 切换为 'python(agent101)'。"
    echo "   (通常位于 Notebook 右上角或 '内核' 菜单中)"
    echo ""
    echo "📚 备用方法 (不推荐): 如果无法切换内核，则必须在**每个**代码单元格的头部重复以下命令:"
    echo ""
    echo "%%script bash"
    echo "# 必须在每个单元格都执行"
    echo "eval \"\$(conda shell.bash hook)\""
    echo "conda activate agent101"
fi

echo "=========================================" 

== Conda 环境检查报告 (仅针对当前 Bash 子进��) ==
✅ 当前单元格已成功激活到 agent101 环境。
✅ 正在使用的环境路径: /root/miniconda3/envs/agent101

💡 提示: 后续的Python单元格将使用Notebook当前选择的Jupyter��核。
   如果需要后续单元格也使用此环境，请执行以下操作:
   1. 检查 Notebook 右上角是否已选择 'python(agent101)'。


In [2]:
# 2. 设置pip 为清华源
%pip config list -v set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
%pip config list -v list


For variant 'global', will try loading '/etc/xdg/pip/pip.conf'
For variant 'global', will try loading '/etc/pip.conf'
For variant 'user', will try loading '/root/.pip/pip.conf'
For variant 'user', will try loading '/root/.config/pip/pip.conf'
For variant 'site', will try loading '/root/miniconda3/envs/agent101/pip.conf'
ERROR: Got unexpected number of arguments, expected 0. (example: "/root/miniconda3/envs/agent101/bin/python -m pip config list")
Note: you may need to restart the kernel to use updated packages.
For variant 'global', will try loading '/etc/xdg/pip/pip.conf'
For variant 'global', will try loading '/etc/pip.conf'
For variant 'user', will try loading '/root/.pip/pip.conf'
For variant 'user', will try loading '/root/.config/pip/pip.conf'
For variant 'site', will try loading '/root/miniconda3/envs/agent101/pip.conf'
ERROR: Got unexpected number of arguments, expected 0. (example: "/root/miniconda3/envs/agent101/bin/python -m pip config list")
Note: you may need to restart th

In [3]:
# 3. 设置HuggingFace代理
%env HF_ENDPOINT=https://hf-mirror.com
# 验证：使用shell命令检查
!echo $HF_ENDPOINT

env: HF_ENDPOINT=https://hf-mirror.com
https://hf-mirror.com


In [4]:
# 🔍 环境信息检查脚本
#
# 本脚本的作用：
# 1. 安装 pandas 库用于数据表格展示
# 2. 检查系统的各项配置信息
# 3. 生成详细的环境报告表格
#
# 对于初学者来说，这个步骤帮助你：
# - 了解当前运行环境的硬件配置
# - 确认是否满足模型运行的最低要求
# - 学习如何通过代码获取系统信息

# 安装 pandas 库 - 用于创建和展示数据表格
# pandas 是 Python 中最流行的数据处理和分析库
%pip install pandas==2.2.2 tabulate==0.9.0

import platform # 导入 platform 模块以获取系统信息
import os # 导入 os 模块以与操作系统交互
import subprocess # 导入 subprocess 模块以运行外部命令
import pandas as pd # 导入 pandas 模块，通常用于数据处理，这里用于创建表格
import shutil # 导入 shutil 模块以获取磁盘空间信息

# 获取 CPU 信息的函数，包括核心数量
def get_cpu_info():
    cpu_info = "" # 初始化 CPU 信息字符串
    physical_cores = "N/A"
    logical_cores = "N/A"

    if platform.system() == "Windows": # 如果是 Windows 系统
        cpu_info = platform.processor() # 使用 platform.processor() 获取 CPU 信息
        try:
            # 获取 Windows 上的核心数量 (需要 WMI)
            import wmi
            c = wmi.WMI()
            for proc in c.Win32_Processor():
                physical_cores = proc.NumberOfCores
                logical_cores = proc.NumberOfLogicalProcessors
        except:
            pass # 如果 WMI 不可用，忽略错误

    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取 CPU 信息和核心数量
        os.environ['PATH'] = os.environ['PATH'] + os.pathsep + '/usr/sbin' # 更新 PATH 环境变量
        try:
            process_brand = subprocess.Popen(['sysctl', "machdep.cpu.brand_string"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_brand, stderr_brand = process_brand.communicate()
            cpu_info = stdout_brand.decode().split(': ')[1].strip() if stdout_brand else "Could not retrieve CPU info"

            process_physical = subprocess.Popen(['sysctl', "hw.physicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_physical, stderr_physical = process_physical.communicate()
            physical_cores = stdout_physical.decode().split(': ')[1].strip() if stdout_physical else "N/A"

            process_logical = subprocess.Popen(['sysctl', "hw.logicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_logical, stderr_logical = process_logical.communicate()
            logical_cores = stdout_logical.decode().split(': ')[1].strip() if stdout_logical else "N/A"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/cpuinfo 文件获取 CPU 信息和核心数量
            with open('/proc/cpuinfo') as f:
                physical_cores_count = 0
                logical_cores_count = 0
                cpu_info_lines = []
                for line in f:
                    if line.startswith('model name'): # 查找以 'model name'开头的行
                        if not cpu_info: # 只获取第一个 model name
                            cpu_info = line.split(': ')[1].strip()
                    elif line.startswith('cpu cores'): # 查找以 'cpu cores' 开头的行
                        physical_cores_count = int(line.split(': ')[1].strip())
                    elif line.startswith('processor'): # 查找以 'processor' 开头的行
                        logical_cores_count += 1
                physical_cores = str(physical_cores_count) if physical_cores_count > 0 else "N/A"
                logical_cores = str(logical_cores_count) if logical_cores_count > 0 else "N/A"
                if not cpu_info:
                     cpu_info = "Could not retrieve CPU info"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    return f"{cpu_info} ({physical_cores} physical cores, {logical_cores} logical cores)" # 返回 CPU 信息和核心数量


# 获取内存信息的函数
def get_memory_info():
    mem_info = "" # 初始化内存信息字符串
    if platform.system() == "Windows":
        # 在 Windows 上不容易通过标准库获取，需要外部库或 PowerShell
        mem_info = "Requires external tools on Windows" # 设置提示信息
    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取内存大小
        process = subprocess.Popen(['sysctl', "hw.memsize"], stdout=subprocess.PIPE, stderr=subprocess.PIPE) # 运行 sysctl 命令
        stdout, stderr = process.communicate() # 获取标准输出和标准错误
        mem_bytes = int(stdout.decode().split(': ')[1].strip()) # 解析输出，获取内存大小（字节）
        mem_gb = mem_bytes / (1024**3) # 转换为 GB
        mem_info = f"{mem_gb:.2f} GB" # 格式化输出
    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/meminfo 文件获取内存信息
            with open('/proc/meminfo') as f:
                total_mem_kb = 0
                available_mem_kb = 0
                for line in f:
                    if line.startswith('MemTotal'): # 查找以 'MemTotal' 开头的行
                        total_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取总内存（KB）
                    elif line.startswith('MemAvailable'): # 查找以 'MemAvailable' 开头的行
                         available_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取可用内存（KB）

                if total_mem_kb > 0:
                    total_mem_gb = total_mem_kb / (1024**2) # 转换为 GB
                    mem_info = f"{total_mem_gb:.2f} GB" # 格式化输出总内存
                    if available_mem_kb > 0:
                        available_mem_gb = available_mem_kb / (1024**2)
                        mem_info += f" (Available: {available_mem_gb:.2f} GB)" # 添加可用内存信息
                else:
                     mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息

        except:
            mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息
    return mem_info # 返回内存信息

# 获取 GPU 信息的函数，包括显存
def get_gpu_info():
    try:
        # 尝试使用 nvidia-smi 获取 NVIDIA GPU 信息和显存
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            gpu_lines = result.stdout.strip().split('\n') # 解析输出，获取 GPU 名称和显存
            gpu_info_list = []
            for line in gpu_lines:
                name, memory = line.split(', ')
                gpu_info_list.append(f"{name} ({memory})") # 格式化 GPU 信息
            return ", ".join(gpu_info_list) if gpu_info_list else "NVIDIA GPU found, but info not listed" # 返回 GPU 信息或提示信息
        else:
             # 尝试使用 lshw 获取其他 GPU 信息 (需要安装 lshw)
            try:
                result_lshw = subprocess.run(['lshw', '-C', 'display'], capture_output=True, text=True)
                if result_lshw.returncode == 0: # 如果命令成功执行
                     # 简单解析输出中的 product 名称和显存
                    gpu_info_lines = []
                    current_gpu = {}
                    for line in result_lshw.stdout.splitlines():
                        if 'product:' in line:
                             if current_gpu:
                                 gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")
                             current_gpu = {'product': line.split('product:')[1].strip()}
                        elif 'size:' in line and 'memory' in line:
                             current_gpu['memory'] = line.split('size:')[1].strip()

                    if current_gpu: # 添加最后一个 GPU 的信息
                        gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")

                    return ", ".join(gpu_info_lines) if gpu_info_lines else "GPU found (via lshw), but info not parsed" # 如果找到 GPU 但信息无法解析，设置提示信息
                else:
                    return "No GPU found (checked nvidia-smi and lshw)" # 如果两个命令都找不到 GPU，设置提示信息
            except FileNotFoundError:
                 return "No GPU found (checked nvidia-smi, lshw not found)" # 如果找不到 lshw 命令，设置提示信息
    except FileNotFoundError:
        return "No GPU found (nvidia-smi not found)" # 如果找不到 nvidia-smi 命令，设置提示信息


# 获取 CUDA 版本的函数
def get_cuda_version():
    try:
        # 尝试使用 nvcc --version 获取 CUDA 版本
        result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            for line in result.stdout.splitlines():
                if 'release' in line: # 查找包含 'release' 的行
                    return line.split('release ')[1].split(',')[0] # 解析行，提取版本号
        return "CUDA not found or version not parsed" # 如果找不到 CUDA 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "CUDA not found" # 如果找不到 nvcc 命令，设置提示信息

# 获取 Python 版本的函数
def get_python_version():
    return platform.python_version() # 获取 Python 版本

# 获取 Conda 版本的函数
def get_conda_version():
    try:
        # 尝试使用 conda --version 获取 Conda 版本
        result = subprocess.run(['conda', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            return result.stdout.strip() # 返回 Conda 版本
        return "Conda not found or version not parsed" # 如果找不到 Conda 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "Conda not found" # 如果找不到 conda 命令，设置提示信息

# 获取物理磁盘空间信息的函数
def get_disk_space():
    try:
        total, used, free = shutil.disk_usage("/") # 获取根目录的磁盘使用情况
        total_gb = total / (1024**3) # 转换为 GB
        used_gb = used / (1024**3) # 转换为 GB
        free_gb = free / (1024**3) # 转换为 GB
        return f"Total: {total_gb:.2f} GB, Used: {used_gb:.2f} GB, Free: {free_gb:.2f} GB" # 格式化输出
    except Exception as e:
        return f"Could not retrieve disk info: {e}" # 如果获取信息出错，设置错误信息

# 获取环境信息
os_name = platform.system() # 获取操作系统名称
os_version = platform.release() # 获取操作系统版本
if os_name == "Linux":
    try:
        # 在 Linux 上尝试获取发行版和版本
        lsb_info = subprocess.run(['lsb_release', '-a'], capture_output=True, text=True)
        if lsb_info.returncode == 0: # 如果命令成功执行
            for line in lsb_info.stdout.splitlines():
                if 'Description:' in line: # 查找包含 'Description:' 的行
                    os_version = line.split('Description:')[1].strip() # 提取描述信息作为版本
                    break # 找到后退出循环
                elif 'Release:' in line: # 查找包含 'Release:' 的行
                     os_version = line.split('Release:')[1].strip() # 提取版本号
                     # 尝试获取 codename
                     try:
                         codename_info = subprocess.run(['lsb_release', '-c'], capture_output=True, text=True)
                         if codename_info.returncode == 0:
                             os_version += f" ({codename_info.stdout.split(':')[1].strip()})" # 将 codename 添加到版本信息中
                     except:
                         pass # 如果获取 codename 失败则忽略

    except FileNotFoundError:
        pass # lsb_release 可能未安装，忽略错误

full_os_info = f"{os_name} {os_version}" # 组合完整的操作系统信息
cpu_info = get_cpu_info() # 调用函数获取 CPU 信息和核心数量
memory_info = get_memory_info() # 调用函数获取内存信息
gpu_info = get_gpu_info() # 调用函数获取 GPU 信息和显存
cuda_version = get_cuda_version() # 调用函数获取 CUDA 版本
python_version = get_python_version() # 调用函数获取 Python 版本
conda_version = get_conda_version() # 调用函数获取 Conda 版本
disk_info = get_disk_space() # 调用函数获取物理磁盘空间信息


# 创建用于存储数据的字典
env_data = {
    "项目": [ # 项目名称列表
        "操作系统",
        "CPU 信息",
        "内存信息",
        "GPU 信息",
        "CUDA 信息",
        "Python 版本",
        "Conda 版本",
        "物理磁盘空间" # 添加物理磁盘空间
    ],
    "信息": [ # 对应的信息列表
        full_os_info,
        cpu_info,
        memory_info,
        gpu_info,
        cuda_version,
        python_version,
        conda_version,
        disk_info # 添加物理磁盘空间信息
    ]
}

# 创建一个 pandas DataFrame
df = pd.DataFrame(env_data)

# 打印表格
print("### 环境信息") # 打印标题
print(df.to_markdown(index=False)) # 将 DataFrame 转换为 Markdown 格式并打印，不包含索引


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
### 环境信息
| 项目         | 信息                                                                               |
|:-------------|:-----------------------------------------------------------------------------------|
| 操作系统     | Linux Ubuntu 22.04.4 LTS                                                           |
| CPU 信息     | 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz (1 physical cores, 4 logical cores) |
| 内存信息     | 5.75 GB (Available: 2.50 GB)                                                       |
| GPU 信息     | No GPU found (nvidia-smi not found)                                                |
| CUDA 信息    | CUDA not found                                                                     |
| Python 版本  | 3.10.18                                                                            |
| Conda 版本   | conda 24.4.0                                         

# OpenAI SDK集成Langfuse获得完整的可观测性
---

## 📚 什么是大模型可观测性？

**可观测性（Observability）** 是监控和调试大语言模型应用的关键技术。想象一下，当你的AI应用在生产环境中运行时，你需要知道：

- 🤔 **模型回答了什么？** - 查看每次对话的完整内容
- ⏱️ **响应速度如何？** - 监控延迟和性能指标  
- 💰 **花费了多少？** - 跟踪Token使用和成本
- 🐛 **哪里出错了？** - 快速定位和解决问题
- 📊 **效果怎么样？** - 评估模型回答质量

**Langfuse** 就是一个专门为大模型应用设计的可观测性平台，它能够：
- 自动记录所有API调用
- 提供直观的可视化界面
- 支持评分和评估
- 帮助优化模型性能

## 🎯 本教程将教会你什么？

1. **基础集成** - 如何用几行代码集成Langfuse
2. **多种调用方式** - 文本、图像、流式、异步调用
3. **高级功能** - 函数调用、评分系统、链路追踪
4. **优秀做法** - 生产环境中的使用技巧


# 🚀 示例手册：OpenAI 集成（Python）

## 为什么需要这个教程？

作为大模型技术初学者，你可能遇到过这些问题：
- 不知道模型到底输出了什么
- 无法追踪API调用的成本
- 调试问题时找不到历史记录
- 不知道如何评估模型效果

**这个教程将彻底解决这些问题！** 通过简单的代码修改，你就能获得企业级的可观测性能力。


## 📖 教程概述

这是一个**零基础友好**的示例手册，演示如何在 Python 项目中集成 Langfuse 与 OpenAI。

### 🔍 Langfuse 能为你做什么？

**Langfuse 会记录每次模型调用的输入输出**，就像给你的AI应用装上了"黑匣子"：

1. **📝 完整记录** - 保存每次对话的完整上下文
2. **🔍 问题排查** - 快速定位错误和异常
3. **📊 质量评估** - 通过评分系统评估模型效果
4. **💰 成本控制** - 监控Token使用和API费用
5. **📈 性能优化** - 分析响应时间和吞吐量

### 🎯 学习目标

完成本教程后，你将掌握：
- ✅ 如何安装和配置Langfuse
- ✅ 如何替换OpenAI SDK实现自动追踪
- ✅ 如何查看和分析追踪数据
- ✅ 如何添加自定义元数据和评分
- ✅ 如何在生产环境中使用

按照 [集成指南](https://langfuse.com/integrations/model-providers/openai-py) 将本集成添加到你的 OpenAI 项目中。


## 🛠️ 环境准备

### 前置知识要求

在开始之前，你需要了解一些基础概念：

- **Python基础** - 变量、函数、类的基本使用
- **API调用** - 了解HTTP请求和响应
- **环境变量** - 如何安全地存储敏感信息
- **Jupyter Notebook** - 基本的notebook操作

### 系统要求

- Python 3.10+
- 稳定的网络连接
- OpenAI API密钥
- Langfuse账户（免费注册）


### 📦 依赖包版本说明

**重要版本要求：**
- **OpenAI SDK** `>=0.27.8` - 基础功能支持
- **OpenAI SDK** `>=1.0.0` - 异步函数和流式输出支持（推荐）

**为什么需要特定版本？**
- 旧版本可能缺少某些功能
- 新版本有更好的性能和稳定性
- 流式输出需要较新的SDK支持

**初学者提示：** 如果你不确定当前版本，直接安装最新版本即可！


In [5]:
# 📥 安装必要的依赖包
# 这个命令会安装两个核心包：
# 1. langfuse - 可观测性平台的核心库
# 2. openai - OpenAI官方SDK

%pip install langfuse==3.3.0 openai==1.107.0

# 💡 初学者提示：
# - %pip 是Jupyter Notebook的魔法命令，用于安装Python包
# - == 指定了确切的版本号，确保环境一致性
# - 如果安装失败，检查网络连接或尝试使用国内镜像源

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [6]:
# 🔐 环境变量配置 - 安全存储敏感信息
# 环境变量是存储API密钥等敏感信息的优秀做法
# 避免在代码中硬编码密钥，防止泄露

import os, getpass

def _set_env(var: str):
    """
    安全地设置环境变量
    如果环境变量不存在，会提示用户输入
    使用getpass模块隐藏输入内容，防止密码泄露
    """
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# 🤖 OpenAI API 配置
# OpenAI API密钥：从 https://platform.openai.com/api-keys 获取
# 这是调用GPT模型必需的认证信息
_set_env("OPENAI_API_KEY")

# API代理地址：如果你使用第三方代理服务（如国内代理）
# 示例：https://api.apiyi.com/v1
# 如果直接使用OpenAI官方API，可以留空
_set_env("OPENAI_BASE_URL")

# # 🌐 Langfuse 配置
# Langfuse是一个可观测性平台，需要注册账户获取密钥
# 注册地址：https://cloud.langfuse.com

# 公开密钥：用于标识你的项目
_set_env("LANGFUSE_PUBLIC_KEY")

# 秘密密钥：用于认证，请妥善保管
_set_env("LANGFUSE_SECRET_KEY")

#  设置基础 URL
# 🇪🇺 欧盟区域(推荐) https://cloud.langfuse.com
# 🇺🇸 美国区域 https://us.cloud.langfuse.com
# 本地测试地址：http://192.168.172.128:3000
_set_env("LANGFUSE_BASE_URL")
# 兼容老版本的基础 URL
_set_env("LANGFUSE_HOST")

# 💡 初学者提示：
# 1. 环境变量存储在操作系统中，重启后需要重新设置
# 2. 生产环境中建议使用.env文件或云服务配置
# 3. 永远不要在代码中硬编码API密钥！

In [7]:
# 🎯 核心集成：替换OpenAI SDK
# 这是整个教程的核心！只需要一行代码的修改
# 只需替换 import 语句，就能用 Langfuse 版本的 OpenAI SDK 获得完整的可观测性

# 原来的导入方式：
# from openai import OpenAI

# 新的导入方式（自动集成Langfuse）：
from langfuse.openai import openai

# 🚀 魔法就在这里！
# Langfuse提供了对原生OpenAI SDK的完全兼容封装
# 这意味着：
# 1. ✅ 你的现有代码无需修改
# 2. ✅ 自动记录所有API调用
# 3. ✅ 保持完全相同的接口
# 4. ✅ 零学习成本

# 💡 初学者理解：
# 想象一下，Langfuse就像一个"透明的中间层"
# 它拦截你的OpenAI调用，记录数据，然后转发给真正的OpenAI API
# 对你来说，使用方式完全一样，但获得了强大的可观测性能力


## 🎯 实战示例

现在让我们通过具体的代码示例来学习如何使用Langfuse！

### 📝 示例1：文本聊天补全

**什么是聊天补全？**
聊天补全是GPT模型的核心功能，它根据你提供的对话历史，生成下一个回复。就像和真人聊天一样！

**应用场景：**
- 智能客服
- 写作助手  
- 代码生成
- 问答系统


In [8]:
# 🧮 简单计算器示例
# 这个例子展示如何让GPT扮演一个精确的计算器

# 发起聊天补全请求
completion = openai.chat.completions.create(
  # 📝 追踪信息
  name="calculator-demo",  # 给这次调用起个名字，方便在Langfuse中查找

  # 🤖 模型选择
  model="gpt-4o",  # 使用GPT-4o模型，你也可以选择gpt-3.5-turbo等

  # 💬 对话内容
  messages=[
      # system消息：定义AI的角色和行为
      {"role": "system", "content": "你是一个非常精确的计算器。你只输出计算结果。"},

      # user消息：用户的实际问题
      {"role": "user", "content": "1 + 1 = "}
  ],

  # 🌡️ 温度控制
  temperature=0,  # 0表示最稳定，适合数学计算等需要准确答案的场景

  # 🏷️ 自定义元数据
  metadata={
      "task_type": "calculator",  # 任务类型
      "difficulty": "easy",       # 难度等级
      "user_id": "demo_user"      # 用户ID
  }
)

# 📊 获取模型响应
# completion.choices[0] 获取第一个（通常也是唯一的）回复
# .message.content 获取回复的文本内容
result = completion.choices[0].message.content
print(f"计算结果: {result}")

# 💡 初学者理解：
# 1. messages是一个对话历史列表，按时间顺序排列
# 2. system消息设置AI的角色，user消息是用户输入
# 3. temperature控制输出的随机性（0-2，越高越随机）
# 4. metadata可以存储任何自定义信息，用于分析和调试


计算结果: 2


![image-20251126153044528](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202511261530111.png)

### 🖼️ 示例2：图像聊天补全

**什么是图像聊天补全？**
GPT-4o等模型不仅能理解文字，还能"看懂"图片！这让AI可以：
- 描述图片内容
- 回答关于图片的问题
- 分析图片中的信息
- 进行图像相关的创作

**应用场景：**
- 图像内容审核
- 视觉问答系统
- 图像标注和描述
- 多模态AI助手


In [9]:
# 🖼️ 图像分析示例
# 这个例子展示如何让GPT分析一张图片

completion = openai.chat.completions.create(
  # 📝 追踪信息
  name="image-analysis-demo",  # 给这次图像分析起个名字

  # 🤖 模型选择（支持视觉的模型）
  model="gpt-4o-mini",  # 支持视觉的模型：GPT-4o、GPT-4o mini、GPT-4 Turbo

  # 💬 多模态对话内容
  messages=[
      # system消息：定义AI在图像分析中的角色
      {"role": "system", "content": "你是一个被训练来描述和解释图像的AI。描述图像中的主要物体和动作。"},

      # user消息：包含文字和图片的复合内容
      {"role": "user", "content": [
        # 文字部分：给AI的指令
        {"type": "text", "text": "这幅画描绘了什么？请详细描述。"},

        # 图片部分：要分析的图片
        {
          "type": "image_url",
          "image_url": {
            "url": "https://static.langfuse.com/langfuse-dev/langfuse-example-image.jpeg"
          },
        },
      ]}
  ],

  # 🌡️ 温度控制
  temperature=0,  # 图像描述需要准确性，使用较低温度

  # 🏷️ 自定义元数据
  metadata={
      "task_type": "image_analysis",
      "image_source": "langfuse_example",
      "analysis_type": "description"
  }
)

# 📊 获取分析结果
analysis_result = completion.choices[0].message.content
print(f"图像分析结果: {analysis_result}")

# 💡 初学者理解：
# 1. 多模态消息：user消息可以同时包含文字和图片
# 2. 图片格式：支持URL链接或base64编码
# 3. 视觉模型：只有特定模型支持图像理解
# 4. Langfuse记录：会自动保存图片URL，方便后续查看


图像分析结果: 这幅画呈现了一个充满未来感和科技感的场景。画面的主色调是深绿色，给人一种神秘和超现实的感觉。画面中央有一个明亮的光束，直冲天际，形成了一个强烈的视觉焦点。这个光束的顶部有一个发光的文字“LANGFUSE”，字母呈现出一种数字化的效果，似乎在闪烁或流动。

在光束的周围，背景中有许多垂直的线条和点状的元素，仿佛是数据流或代码在不断地下落，营造出一种信息爆炸的感觉。这些线条和点的排列方式使得整个画面看起来像是一个虚拟世界或数字空间，给人一种身处于网络或计算机系统中的错觉。

画面的底部有一些模糊的轮廓，可能是城市的轮廓或其他结构，进一步增强了未来科技的氛围。整体而言，这幅画通过其色彩、光线和构图，传达了一种对科技和数字世界的探索与思考。


![image-20251126153325118](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202511261533566.png)

#### 🔍 查看追踪结果

现在你可以前往 [Langfuse控制台](https://cloud.langfuse.com) 查看刚才的API调用记录！

**在Langfuse中你可以看到：**
- 📊 **完整的对话历史** - 包括system和user消息
- ⏱️ **响应时间** - 了解API性能
- 💰 **Token使用量** - 监控成本
- 🖼️ **图片链接** - 方便复现图像分析

### 🌊 示例3：流式聊天补全

**什么是流式输出？**
流式输出让AI可以"边想边说"，就像真人对话一样逐字逐句地回复，而不是等待完整答案。

**流式输出的优势：**
- ⚡ **更快的响应感知** - 用户立即看到AI开始回复
- 🎯 **更好的用户体验** - 避免长时间等待
- 🔄 **实时交互** - 可以中途停止或修改
- 📱 **适合移动端** - 减少网络超时风险

**应用场景：**
- 聊天机器人
- 长文本生成
- 实时翻译
- 代码生成助手


In [10]:
# 🎭 流式笑话生成示例
# 这个例子展示如何实时获取AI的回复

print("🤖 AI开始讲笑话：")
print("-" * 40)

# 发起流式聊天补全请求
completion = openai.chat.completions.create(
  # 📝 追踪信息
  name="streaming-joke-demo",  # 给这次流式调用起个名字

  # 🤖 模型选择
  model="gpt-4o",

  # 💬 对话内容
  messages=[
      {"role": "system", "content": "你是一位专业的喜剧演员，擅长讲有趣的笑话。"},
      {"role": "user", "content": "讲一个关于编程的笑话给我听。"}
  ],

  # 🌡️ 温度控制
  temperature=0.7,  # 稍微提高温度，让笑话更有创意

  # 🏷️ 自定义元数据
  metadata={
      "task_type": "joke_generation",
      "style": "programming_humor",
      "streaming": True
  },

  # 🌊 开启流式模式
  stream=True  # 这是关键！开启后API会边生成边返回
)

# 📊 处理流式响应
# completion现在是一个生成器，会逐步返回内容
for chunk in completion:
    # 检查chunk中是否有新内容
    if chunk.choices[0].delta.content is not None:
        # 实时打印内容，end=""表示不换行
        print(chunk.choices[0].delta.content, end="", flush=True)

print("\n" + "-" * 40)
print("✅ 笑话讲完了！")

# 💡 初学者理解：
# 1. stream=True 开启流式模式
# 2. 返回的是生成器，需要用for循环处理
# 3. delta.content 包含新增的内容片段
# 4. flush=True 确保内容立即显示
# 5. Langfuse会自动记录整个流式过程


🤖 AI开始讲笑话：
----------------------------------------
当然啦！听好了：

为什么程序员喜欢黑咖啡？  
因为喝咖啡的时候，他们不需要加任何“类”！
----------------------------------------
✅ 笑话讲完了！


![image-20251126153933013](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202511261539373.png)

### ⚡ 示例4：异步聊天补全

**什么是异步调用？**
异步调用允许程序在等待API响应时继续执行其他任务，提高程序的并发性能。

**异步的优势：**
- 🚀 **更高的并发性** - 同时处理多个请求
- ⏱️ **更好的资源利用** - 避免阻塞等待
- 📈 **更高的吞吐量** - 适合高并发场景
- 🔄 **非阻塞操作** - 不会卡住整个程序

**应用场景：**
- 批量处理大量请求
- 微服务架构
- 高并发Web应用
- 实时数据处理


In [11]:
# ⚡ 异步客户端初始化
# 异步客户端允许非阻塞的API调用，提高程序性能

from langfuse.openai import AsyncOpenAI

# 创建异步客户端实例
# 自动复用环境变量中的Langfuse配置
async_client = AsyncOpenAI()

# 💡 初学者理解：
# 1. AsyncOpenAI 是异步版本的OpenAI客户端
# 2. 使用方式与同步客户端基本相同
# 3. 主要区别是需要使用 await 关键字
# 4. 适合需要并发处理多个请求的场景


In [12]:
# 🧮 异步计算器示例
# 这个例子展示如何使用异步客户端进行API调用

# 在异步函数内调用聊天补全接口
completion = await async_client.chat.completions.create(
  # 📝 追踪信息
  name="async-calculator-demo",  # 为本次异步调用命名

  # 🤖 模型选择
  model="gpt-4o",

  # 💬 对话内容
  messages=[
      {"role": "system", "content": "你是一个非常精确的计算器。你只输出计算结果。"},
      {"role": "user", "content": "1 + 100 = "}
  ],

  # 🌡️ 温度控制
  temperature=0,

  # 🏷️ 自定义元数据
  metadata={
      "task_type": "async_calculator",
      "concurrency": "high",
      "user_id": "async_demo_user"
  }
)

# 📊 获取异步响应结果
result = completion.choices[0].message.content
print(f"异步计算结果: {result}")

# 💡 初学者理解：
# 1. await 关键字用于等待异步操作完成
# 2. 异步调用不会阻塞程序执行
# 3. Langfuse自动处理异步调用的追踪
# 4. 适合需要同时处理多个请求的场景


异步计算结果: 101


![image-20251126154122496](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202511261541951.png)

### 🔧 示例5：函数调用（Function Calling）

**什么是函数调用？**
函数调用让AI可以调用你定义的函数，实现更复杂的交互和结构化输出。

**函数调用的优势：**
- 🎯 **结构化输出** - 获得格式化的JSON数据
- 🔗 **外部集成** - 调用数据库、API等外部服务
- 📊 **数据验证** - 使用Pydantic确保数据格式正确
- 🤖 **智能决策** - AI根据情况选择调用哪个函数

**应用场景：**
- 智能助手（查询天气、发送邮件）
- 数据提取和转换
- 工作流自动化
- API接口生成


In [13]:
# 📦 安装Pydantic依赖
# Pydantic是一个强大的数据验证库，用于定义结构化数据模型

%pip install pydantic==2.11.9

# 💡 初学者理解：
# Pydantic帮助我们：
# 1. 定义数据结构（类似数据库表结构）
# 2. 自动验证数据格式
# 3. 生成JSON Schema供AI使用
# 4. 确保AI返回的数据符合预期格式

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [14]:
from typing import List
from pydantic import BaseModel

# 定义函数调用返回值的数据结构，让模型生成结构化的 JSON
class StepByStepAIResponse(BaseModel):
    title: str  # 标题：例如“装机步骤”
    steps: List[str]  # 步骤列表：每个元素是一句描述

schema = StepByStepAIResponse.schema()  # 返回 JSON Schema，供 OpenAI 函数调用使用


/tmp/ipykernel_216832/414262564.py:9: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  schema = StepByStepAIResponse.schema()  # 返回 JSON Schema，供 OpenAI 函数调用使用


In [15]:
import json

# 示例：引导模型调用我们定义的函数，并返回结构化结果
response = openai.chat.completions.create(
    name="test-function",
    model="gpt-4o",  # 支持函数调用的模型版本(有些OpenAI代理不支持函数调用，这里可以替换为DeepSeek)
    messages=[
       {"role": "user", "content": "如何组装一台电脑"}
    ],
    functions=[
        {
          "name": "get_answer_for_user_query",  # 函数名称，需要与业务代码保持一致
          "description": "分步骤为用户提供答案",  # 告诉模型函数的用途
          "parameters": StepByStepAIResponse.schema()  # Pydantic 自动生成的参数定义
        }
    ],
    function_call={"name": "get_answer_for_user_query"}  # 强制模型调用指定函数
)

# Langfuse 会记录函数调用的入参与出参，便于追踪
output = json.loads(response.choices[0].message.function_call.arguments)  # 将字符串反序列化为 Python 字典


/tmp/ipykernel_216832/539520121.py:14: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  "parameters": StepByStepAIResponse.schema()  # Pydantic 自动生成的参数定义


前往 https://cloud.langfuse.com 或你自建的实例，可以在 Langfuse 中查看生成记录。
![image-20251126154233516](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202511261542923.png)


## Langfuse 功能（用户、标签、元数据、会话）

你可以在 OpenAI 请求中加入额外属性，以启用更多 Langfuse 功能。Langfuse 集成会自动解析这些字段。完整功能列表见 [文档](https://langfuse.com/integrations/model-providers/openai-py#custom-trace-properties)。


In [16]:
result = openai.chat.completions.create(
    name="test-chat-with-attributes",  # trace 名称，对应 Langfuse 中的 Trace.name
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "你是一个非常精确的计算器。你只输出计算结果。"},
        {"role": "user", "content": "1 + 1 = "}],
    temperature=0,
    metadata={
        "langfuse_session_id": "session_123", # 会话 ID，用于区分不同对话/请求
        "langfuse_user_id": "user_456", # 业务用户 ID，让你在 Langfuse 中按用户聚合
        "langfuse_tags": ["calculator"], # trace 标签，可用于 Langfuse 控制台筛选
        "someMetadataKey": "someValue"  # trace 元数据，适合记录业务上下文
    }
)


示例追踪：
![image-20251126154541203](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202511261545581.png)

## 将多次生成归并为单个 Trace

在实际应用中，往往需要多次调用 OpenAI。借助 `@observe()` 装饰器，可以把一次 API 调用中的所有 LLM 请求归入 Langfuse 中同一个 `trace`。


In [17]:
from langfuse.openai import openai
from langfuse import observe

# 【@observe 装饰器】会自动：
# 1. 为 main 函数创建一个顶层 trace
# 2. 捕获函数内部的所有 Langfuse/OpenAI 调用，并将它们串联为一个完整链路
@observe()  # 装饰器会自动创建 trace 并嵌套各次生成
def main(country: str, user_id: str, **kwargs) -> str:
    # 嵌套调用 1：询问国家首都
    capital = openai.chat.completions.create(
      name="geography-teacher",
      model="gpt-4o",
      messages=[
          {"role": "system", "content": "你是一位地理老师，帮助学生学习国家的首都。当被问及时，只输出首都。"},
          {"role": "user", "content": country}],
      temperature=0,
    ).choices[0].message.content  # 读取模型回复

    # 嵌套调用 2：请模型写一首关于首都的诗
    poem = openai.chat.completions.create(
      name="poet",
      model="gpt-4o",
      messages=[
          {"role": "system", "content": "你是一位诗人。创作一首关于城市的诗。"},
          {"role": "user", "content": capital}],
      temperature=1,  # 提高温度，让诗歌更有创意
      max_tokens=200,  # 控制输出长度
    ).choices[0].message.content

    return poem

# 直接调用 main 函数，Langfuse 会自动生成 trace 并可在控制台查看链路
print(main("北京", "FLY"))


### 城市之魂

高楼穿云，苍穹低头，  
人潮似海，喧嚣如流。  
霓虹闪烁映心梦，  
街巷深处故事悠。

古墙依旧镌刻岁月，  
新桥横架时代洪流。  
小贩吆喝，市场喧闹，  
摩天大厦起舞风柔。

赤色晨光披黄土地，  
江河奔腾载万物舟。  
车轮滚滚穿梭疾速，  
电光交汇织梦无休。

此间有诗，古韵悠悠，  
纸墨铺陈千载长轴；  
此间有心，铁骨柔肠，  
一城一魂一国昂首。

城市不眠，烟火常在，  
繁华深处有乡愁徘徊。  
这是中国


前往 https://cloud.langfuse.com 或你自建的实例，可以在 Langfuse 中查看完整链路。

![多次 OpenAI 调用的追踪图](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202511261546555.png)


## 完整方案：与 Langfuse SDK 协同

`trace` 是 Langfuse 的核心对象，你可以为它附加丰富的元数据。详见 [Python SDK 文档](https://langfuse.com/docs/sdk/python#traces-1)。

自定义 trace 后可以实现以下能力：
- 自定义名称，用来区分不同类型的链路
- 以用户为粒度的追踪
- 通过版本与发布信息进行实验管理
- 保存自定义元数据


In [18]:
from langfuse.openai import openai
from langfuse import observe, get_client

langfuse = get_client()  # 获取底层 Langfuse 客户端，可在装饰器之外手动操作 trace

@observe()  # 装饰器会自动创建 trace 并嵌套各次生成
def main(country: str, user_id: str, **kwargs) -> str:
    # 嵌套调用 1：获取国家首都
    capital = openai.chat.completions.create(
      name="geography-teacher",
      model="gpt-4o",
      messages=[
          {"role": "system", "content": "你是一位地理老师，帮助学生学习国家的首都。当被问及时，只输出首都。"},
          {"role": "user", "content": country}],
      temperature=0,
    ).choices[0].message.content

    # 嵌套调用 2：根据首都生成诗歌
    poem = openai.chat.completions.create(
      name="poet",
      model="gpt-4o",
      messages=[
          {"role": "system", "content": "你是一位诗人。创作一首关于城市的诗。"},
          {"role": "user", "content": capital}],
      temperature=1,
      max_tokens=200,
    ).choices[0].message.content

    # 手动更新当前 trace 的属性，让仪表盘信息更完整
    langfuse.update_current_trace(
        name="City poem generator",  # 自定义 trace 名称
        session_id="1234",  # 业务会话 ID
        user_id=user_id,  # 业务用户 ID
        tags=["tag1", "tag2"],  # 标签，支持在 Langfuse 中搜索
        public=True,  # 是否允许分享 Trace 链接
        metadata = {"env": "development"}  # 自定义元数据，例如环境标记
    )

    return poem

# create_trace_id() 会生成一个可复用的追踪 ID，你也可以改为自己的业务 ID
trace_id = langfuse.create_trace_id()

# 通过关键字参数 `langfuse_observation_id` 将 trace_id 传递给装饰器，方便串联上下游调用
print(main("北京", "admin", langfuse_observation_id=trace_id))


**城市梦境**  

霓虹闪烁，街道如河，  
人群汇聚，化作奔流的歌。  
高楼直立，接天欲高，  
钢筋水泥，如画般的荣耀。  

城中的巷弄，承载古老的回声，  
竹影婆娑，仍记千年的清风。  
小摊烟火，飘散记忆的香味，  
繁华深处，有生活的安慰。  

白昼喧嚣，工厂的脉搏跳动，  
夜晚静谧，繁星藏在心中。  
城市的心，是千万步伐交织，  
人人向前，追逐不尽的故事。  

高铁呼啸，连通四方的梦境，  
城市的脉络如盛开的锦屏。  
游人踏过古城门，


示例：
![image-20251126154814338](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202511261548627.png)

## 以编程方式添加评分


你可以向 trace 添加 [评分](https://langfuse.com/docs/scores)，记录用户反馈或自动化评估结果。评分可用于在 Langfuse 中筛选追踪，并会显示在控制台中。详见评分文档。

评分通过 `trace_id` 与对应的 trace 关联。


In [19]:
from langfuse import observe, get_client

langfuse = get_client()  # 获取底层客户端，用于主流程之外的操作

@observe()  # 装饰器会自动创建 trace 并嵌套各次生成
def main():
    # 在装饰器内部，可随时获取当前 trace 的 ID
    trace_id = langfuse.get_current_trace_id()

    # TODO: 在此处编写你的业务逻辑，例如继续调用其他 API、处理用户输入等

    return "res", trace_id

# 执行被装饰的函数，Langfuse 会生成 trace
_, trace_id = main()

# 在 trace 上下文外部也可以继续操作，例如向这次 trace 添加评分
langfuse.create_score(
    trace_id=trace_id,  # 指定要打分的 trace
    name="my-score-name",  # 评分名称，用于区分不同指标
    value=1  # 分值，可以是布尔、整数、浮点数，视业务场景而定
)


示例：![image-20251126154926071](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202511261549411.png)